In [2]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import warnings


In [7]:
dataset=pd.read_csv(r"training_set.tsv",delimiter="\t",encoding="ISO-8859-1")
dataset.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        12976 non-null  int64  
 1   essay_set       12976 non-null  int64  
 2   essay           12976 non-null  object 
 3   rater1_domain1  12976 non-null  int64  
 4   rater2_domain1  12976 non-null  int64  
 5   rater3_domain1  128 non-null    float64
 6   domain1_score   12976 non-null  int64  
 7   rater1_domain2  1800 non-null   float64
 8   rater2_domain2  1800 non-null   float64
 9   domain2_score   1800 non-null   float64
 10  rater1_trait1   2292 non-null   float64
 11  rater1_trait2   2292 non-null   float64
 12  rater1_trait3   2292 non-null   float64
 13  rater1_trait4   2292 non-null   float64
 14  rater1_trait5   723 non-null    float64
 15  rater1_trait6   723 non-null    float64
 16  rater2_trait1   2292 non-null   float64
 17  rater2_trait2   2292 non-null  

In [4]:
dataset.isnull().sum()


essay_id              0
essay_set             0
essay                 0
rater1_domain1        0
rater2_domain1        0
rater3_domain1    12848
domain1_score         0
rater1_domain2    11176
rater2_domain2    11176
domain2_score     11176
rater1_trait1     10684
rater1_trait2     10684
rater1_trait3     10684
rater1_trait4     10684
rater1_trait5     12253
rater1_trait6     12253
rater2_trait1     10684
rater2_trait2     10684
rater2_trait3     10684
rater2_trait4     10684
rater2_trait5     12253
rater2_trait6     12253
rater3_trait1     12848
rater3_trait2     12848
rater3_trait3     12848
rater3_trait4     12848
rater3_trait5     12848
rater3_trait6     12848
dtype: int64

In [9]:
dataset.dropna(axis=1, inplace=True)
dataset.drop(["rater1_domain1", "rater2_domain1"], axis=1, inplace=True)
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   essay_id       12976 non-null  int64 
 1   essay_set      12976 non-null  int64 
 2   essay          12976 non-null  object
 3   domain1_score  12976 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 405.6+ KB


In [10]:
essay_domainScore = dataset[["essay_set", "domain1_score"]].groupby("essay_set").describe()["domain1_score"][["min", "max"]]
essay_domainScore.head(50)

,min,max
essay_set,,
1,2.0,12.0
2,1.0,6.0
3,0.0,3.0
4,0.0,3.0
5,0.0,4.0
6,0.0,4.0
7,2.0,24.0
8,10.0,60.0


In [11]:
dataset['essay_set'].value_counts()

5    1805
2    1800
6    1800
1    1783
4    1770
3    1726
7    1569
8     723
Name: essay_set, dtype: int64

In [9]:
dataset.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [10]:
print(min(dataset['domain1_score']))
print(max(dataset['domain1_score']))


0
60


In [12]:
min_range = [2,1,0,0,0,0,0,0]
max_range = [12,6,3,3,4,4,30,60]

def normalize(x,mi,ma):
    #print("Before Normalization: "+str(x))
    x = (x-mi)/(ma-mi)
    #print("After Normalization : "+str(x))
    return round(x*10)

dataset['final_score']=dataset.apply(lambda x:normalize(x['domain1_score'],min_range[x['essay_set']-1],max_range[x['essay_set']-1]),axis=1)

In [13]:
dataset.head()

,essay_id,essay_set,essay,domain1_score,final_score
0,1,1,"Dear local newspaper, I think effects computer...",8,6
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,7
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,8
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,6


In [15]:
print(min(dataset['final_score']))
print(max(dataset['final_score']))


0
10


In [17]:
dataset.drop('domain1_score',axis=1,inplace=True)
dataset.head()

,essay_id,essay_set,essay,final_score
0,1,1,"Dear local newspaper, I think effects computer...",6
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8
4,5,1,"Dear @LOCATION1, I know having computers has a...",6


# Preprocessing and cleaning data

In [14]:
def clean_essay(essay):
    words=[]
    for i in essay.split():
        if i.startswith("@"):
            continue
        else:
            words.append(i);
    return ' '.join(words)
dataset['essay'] = dataset['essay'].apply(lambda x:clean_essay(x))
dataset['essay']

0        Dear local newspaper, I think effects computer...
1        Dear I believe that using computers will benef...
2        Dear, More and more people use computers, but ...
3        Dear Local Newspaper, I have found that many e...
4        Dear I know having computers has a positive ef...
                               ...                        
12971    In most stories mothers and daughters are eith...
12972    I never understood the meaning laughter is the...
12973    When you laugh, is out of habit, or is cause? ...
12974    Trippin' on fences I am years young, and in th...
12975    Many people believe that laughter can improve ...
Name: essay, Length: 12976, dtype: object

In [15]:
stop_words=set(stopwords.words('english'))
def remove_stop_words(essay):
    word_tokens=word_tokenize(essay)
    clean_sentence=[]
    for w in word_tokens:
        if w not in stop_words:
            clean_sentence.append(w)
    return ' '.join(clean_sentence)
dataset['cleaned_essay']=dataset['essay'].apply(lambda x:remove_stop_words(x))
dataset.head()

,essay_id,essay_set,essay,domain1_score,final_score,cleaned_essay
0,1,1,"Dear local newspaper, I think effects computer...",8,6,"Dear local newspaper , I think effects compute..."
1,2,1,Dear I believe that using computers will benef...,9,7,Dear I believe using computers benefit us many...
2,3,1,"Dear, More and more people use computers, but ...",7,5,"Dear , More people use computers , everyone ag..."
3,4,1,"Dear Local Newspaper, I have found that many e...",10,8,"Dear Local Newspaper , I found many experts sa..."
4,5,1,Dear I know having computers has a positive ef...,8,6,Dear I know computers positive effect people ....


In [16]:
def remove_punct(essay):
    essay = re.sub("[^A-Za-z ]","",essay)
    return essay
dataset['cleaned_essay'] = dataset['cleaned_essay'].apply(lambda x:remove_punct(x))
dataset.head()


,essay_id,essay_set,essay,domain1_score,final_score,cleaned_essay
0,1,1,"Dear local newspaper, I think effects computer...",8,6,Dear local newspaper I think effects computer...
1,2,1,Dear I believe that using computers will benef...,9,7,Dear I believe using computers benefit us many...
2,3,1,"Dear, More and more people use computers, but ...",7,5,Dear More people use computers everyone agre...
3,4,1,"Dear Local Newspaper, I have found that many e...",10,8,Dear Local Newspaper I found many experts say...
4,5,1,Dear I know having computers has a positive ef...,8,6,Dear I know computers positive effect people ...


In [99]:
def sentence_to_words(sentence):
    sentence=re.sub("[^A-Za-z0-9]"," ",sentence)
    words=nltk.word_tokenize(sentence)
    return words

def essay_to_sentence(essay):
    essay=essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay) #Converts essay into tokens of sentences
    words=[]
    for i in raw:
        if(len(i)>0): #If sentence length is greater than 0
            words.append(sentence_to_words(i))
    return words
    
    
def char_count(essay):
    charcount=0
    for i in essay_to_sentence(essay):
        for j in i:
            charcount=charcount+len(j)
    return charcount
    

def word_count(essay):
    wordcount=0
    for i in essay_to_sentence(essay):
        wordcount=wordcount+len(i)
    return wordcount
    
def avg_word_length(essay):
    return char_count(essay)/word_count(essay)
    

data = open('big.txt').read()
words = re.findall('[a-z]+', data.lower())  
def spelling_mistakes(essay):
    essay=essay.lower()
    new_essay = re.sub("[^A-Za-z0-9]"," ",essay)
    new_essay = re.sub("[0-9]","",new_essay)
    count=0
    all_words = new_essay.split()
    for i in all_words:
        if i not in words:
            count+=1
    return count


def sentences_count(essay):
    return len(essay_to_sentence(essay))


def grammar_count(essay):
    sentences = essay_to_sentence(essay)
    noun_count=0
    adj_count=0
    verb_count=0
    adverb_count=0
    for i in sentences:
        pos_sentence = nltk.pos_tag(i)
        for j in pos_sentence:
            pos_tag = j[1]
            if(pos_tag[0]=='N'):
                noun_count+=1
            elif(pos_tag[0]=='V'):
                verb_count+=1
            elif(pos_tag[0]=='J'):
                adj_count+=1
            elif(pos_tag[0]=='R'):
                adverb_count+=1
    return noun_count,verb_count,adj_count,adverb_count
    


In [100]:
dataset2=dataset.copy()
dataset2 = dataset.copy()
dataset2['char_count'] = dataset2['essay'].apply(char_count)
print("Done with character count")
dataset2['word_count'] = dataset2['essay'].apply(word_count)
print("Done with word count")

dataset2['sentence_count'] = dataset2['essay'].apply(sentences_count)
print("Done with sentence count")


dataset2['avg_word_len'] = dataset2['essay'].apply(avg_word_length)
print("Done with avg word len count")

dataset2['spelling_mistake_count'] = dataset2['essay'].apply(spelling_mistakes)
print("Done with spelling msitake count")


dataset2['noun_count'], dataset2['adj_count'], dataset2['verb_count'], dataset2['adv_count'] = zip(*dataset2['essay'].map(grammar_count))
print("Done")

dataset2.to_csv("Final_Data.csv")
dataset2.head()


Done with character count
Done with word count
Done with sentence count
Done with avg word len count
Done with spelling msitake count
Done


,essay_id,essay_set,essay,final_score,cleaned_essay,char_count,word_count,sentence_count,avg_word_len,spelling_mistake_count,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24
1,2,1,Dear I believe that using computers will benef...,7,Dear I believe using computers benefit us many...,1765,413,17,4.273608,21,98,84,20,19
2,3,1,"Dear, More and more people use computers, but ...",5,Dear More people use computers everyone agre...,1185,276,14,4.293478,5,76,51,20,16
3,4,1,"Dear Local Newspaper, I have found that many e...",8,Dear Local Newspaper I found many experts say...,2284,490,26,4.661224,31,142,96,39,29
4,5,1,Dear I know having computers has a positive ef...,6,Dear I know computers positive effect people ...,2023,469,30,4.313433,18,110,90,32,36


In [101]:
dataset2.to_csv("FormattedEssaysWithScore.csv")


In [4]:
formatted_dataset=pd.read_csv("FormattedEssaysWithScore.csv")
formatted_dataset.drop('Unnamed: 0',inplace=True,axis=1)
formatted_dataset.head()

,essay_id,essay_set,essay,final_score,cleaned_essay,char_count,word_count,sentence_count,avg_word_len,spelling_mistake_count,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24
1,2,1,Dear I believe that using computers will benef...,7,Dear I believe using computers benefit us many...,1765,413,17,4.273608,21,98,84,20,19
2,3,1,"Dear, More and more people use computers, but ...",5,Dear More people use computers everyone agre...,1185,276,14,4.293478,5,76,51,20,16
3,4,1,"Dear Local Newspaper, I have found that many e...",8,Dear Local Newspaper I found many experts say...,2284,490,26,4.661224,31,142,96,39,29
4,5,1,Dear I know having computers has a positive ef...,6,Dear I know computers positive effect people ...,2023,469,30,4.313433,18,110,90,32,36


In [5]:
formatted_dataset.shape

(12976, 14)

In [17]:
vectorizer = CountVectorizer(max_features = 10000, ngram_range=(1, 3), stop_words='english')
count_vectors = vectorizer.fit_transform(dataset['cleaned_essay'])
feature_names = vectorizer.get_feature_names()
data = dataset[['essay_set','cleaned_essay','final_score']].copy()
X = count_vectors.toarray()
y = data['final_score'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [18]:
y_train.shape

(9083,)

Linear Regression

In [32]:
#Save Trained Model
linear_regressor = LinearRegression()
model=linear_regressor.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))

Mean squared error: 200260978288845.22


In [34]:
clf = SVR(C=1.0, epsilon=0.2)
model=clf.fit(X_train, y_train)
y_pred=model.predict(X_test)
print("Mean squared error:%.2f"%mean_squared_error(y_test,y_pred))

Mean squared error:2.27


In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
model=rf.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))